In [1]:
%load_ext autoreload
%autoreload 2
import sys, os
from os.path import expanduser
## actions required!!!!!!!!!!!!!!!!!!!! change your folder path 
path_repo = expanduser("~/Documents/G3_2/regime-identification"); sys.path.append(path_repo)
path_file = expanduser("~/data/G3_2/regime-identification/simulation")
path = {}
for folder in ["data", "estimation", "score", "figure", "latex"]:
    path[folder] = f"{path_file}/{folder}"

In [2]:
from hmmlearn.base import BaseHMM

In [18]:
from regime.simulation_helper import *

In [3]:
import numpy as np
from numpy.random import RandomState

In [4]:
rng = RandomState(0)

In [6]:
dof = 5

In [11]:
Xs = rng.standard_t(dof, size=(10000))

In [12]:
Xs.std()

1.2734326189102665

In [13]:
np.sqrt(5/3)

1.2909944487358056

In [29]:
class tHMM(BaseHMM):
    def __init__(self, n_components, random_state):
        super().__init__(n_components=n_components, random_state=random_state,)
    
    def _check(self):
        return 
    
    def _generate_sample_from_state(self, state, random_state):
        """
        Generate a random sample from a given component.
        Parameters
        ----------
        state : int
            Index of the component to condition on.
        random_state: RandomState
            A random number generator instance.  (`sample` is the only caller
            for this method and already normalizes *random_state*.)
        Returns
        -------
        X : array, shape (n_features, )
            A random sample from the emission distribution corresponding
            to a given component.
        """
        return self.means_[state, 0] + np.sqrt(self.covars_[state, 0] * (self.dof_-2)/self.dof_) * random_state.standard_t(self.dof_)    

In [50]:
def get_tHMM_model(means_, covars_, transmat_, dof_, startprob_=None, random_state=None):
    """
    get a GaussianHMM model with the given params. The instance won't update any parameter if fit method is called. 
    """
    n_c, n_f = means_.shape
    random_state = check_random_state(random_state)
    
    if startprob_ is None:
        startprob_ = invariant_dist_transmat_(transmat_)
        
    hmm_model = tHMM(n_components=n_c, random_state=random_state)
    # hmm_model.n_features = n_f
    hmm_model.dof_ = dof_
    hmm_model.means_ = means_
    hmm_model.covars_ = covars_
    hmm_model.transmat_ = transmat_
    hmm_model.startprob_ = startprob_
    return hmm_model

In [51]:
thmm_model = get_tHMM_model(*load_hardy_params("daily"), 5)

In [30]:
thmm_model = tHMM(2, rng)

In [31]:
thmm_model.means_, thmm_model.covars_, thmm_model.transmat_ = load_hardy_params("daily")

In [32]:
thmm_model.dof_=5

In [34]:
from tqdm import trange

In [44]:
res = []
for i in trange(100000):
    res.append(thmm_model._generate_sample_from_state(1, thmm_model.random_state))

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100000/100000 [00:00<00:00, 755509.92it/s]


In [47]:
np.array(res).mean()

-0.0007372555042821692

In [48]:
thmm_model.means_

array([[ 0.000615],
       [-0.000785]])

In [46]:
np.sqrt(thmm_model.covars_)

array([[0.00775916],
       [0.01739661]])

In [33]:
thmm_model._generate_sample_from_state(0, thmm_model.random_state)

0.008659769253273074

In [19]:
load_hardy_params("daily")

(array([[ 0.000615],
        [-0.000785]]),
 array([[6.02045e-05],
        [3.02642e-04]]),
 array([[0.99788424, 0.00211576],
        [0.01198171, 0.98801829]]))

In [ ]:
    def _generate_sample_from_state(self, state, random_state):
        return random_state.multivariate_normal(
            self.means_[state], self.covars_[state]
        )
